In [1]:
import pandas as pd
import datetime
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates

import chart_studio.plotly as py
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

import altair as alt

# Make Plotly work in your Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# Use Plotly locally
cf.go_offline()

%matplotlib inline

In [3]:
url = 'https://raw.githubusercontent.com/ngophuong/VNStock/main/excel_shs.csv'
df = pd.read_csv(url, index_col = 0)
df['datetime']=pd.to_datetime(df['<DTYYYYMMDD>'],format='%Y%m%d')
df['Month_no']= df['datetime'].dt.month
df['Year_no']= df['datetime'].dt.year
df['Period'] = df['Year_no'].astype(str) + str("-") + df['Month_no'].astype(str)
df.head(3)

,<DTYYYYMMDD>,<OpenFixed>,<HighFixed>,<LowFixed>,<CloseFixed>,<Volume>,<Open>,<High>,<Low>,<Close>,<VolumeDeal>,<VolumeFB>,<VolumeFS>,datetime,Month_no,Year_no,Period
<Ticker>,,,,,,,,,,,,,,,,,
SHS,20210429,27.4,27.8,27.2,27.4,4550766,27.4,27.8,27.2,27.4,0,100,4200,2021-04-29,4,2021,2021-4
SHS,20210428,26.9,27.9,26.7,27.3,4763896,26.9,27.9,26.7,27.3,0,100,5600,2021-04-28,4,2021,2021-4
SHS,20210427,26.9,27.3,26.5,26.9,4020723,26.9,27.3,26.5,26.9,40000,11000,30000,2021-04-27,4,2021,2021-4


In [5]:
df =df[['<Close>','datetime','<Volume>','Period','Year_no']]
df = df.head(1500)

In [19]:
#plot linechart for stock price
short_rolling = df['<Close>'].rolling(window=20).mean()

ema_long = df['<Close>'].ewm(span=50, adjust=False).mean()
ema_short = df['<Close>'].ewm(span=20, adjust=False).mean()
line_price = df['<Close>']
fig = go.Figure()
#px.line(df_hpg, x='datetime', y='<Close>', labels={'x':'Date', 'y':'Price'})
fig.update_layout(  autosize=False,   width=1200,    height=800)
fig.add_trace(go.Scatter(x=df.datetime, y=line_price, 
                        mode='lines', name='SHS'))

fig.add_trace(go.Scatter(x=df.datetime, y=ema_short, 
                        mode='lines', name='Span 20-days EMA'))

fig.add_trace(go.Scatter(x=df.datetime, y=ema_long, 
                        mode='lines', name='Span 50-days EMA'))


In [5]:
#px.violin(df_tips, y="tip", x="smoker", color="sex", box=True, points="all",
#          hover_data=df_tips.columns)

fig = go.Figure()
px.violin(df,x='Period',y='<Close>',box =True, points='all',
         hover_data=df.columns, title = 'STOCK Violin chart', height= 800, width= 1200)

In [6]:
df.tail(3)

,<DTYYYYMMDD>,<OpenFixed>,<HighFixed>,<LowFixed>,<CloseFixed>,<Volume>,<Open>,<High>,<Low>,<Close>,<VolumeDeal>,<VolumeFB>,<VolumeFS>,datetime,Month_no,Year_no,Period
<Ticker>,,,,,,,,,,,,,,,,,
SHS,20090629,15.3947,15.9663,15.1661,15.3567,1534300,40.4,41.9,39.8,40.3,0,0,0,2009-06-29,6,2009,2009-6
SHS,20090626,15.8140,16.0045,15.1662,15.1661,1163400,41.5,42.0,39.8,39.8,0,500,0,2009-06-26,6,2009,2009-6
SHS,20090625,14.7089,16.3856,13.7181,15.3186,1828600,38.6,43.0,36.0,40.2,30000,202500,0,2009-06-25,6,2009,2009-6


In [7]:
#HDFC_df[‘Day_Perc_Change’] = HDFC_df[‘Adj Close’].pct_change()*100
df['Daily_return']=df['<Close>'].pct_change()*100
df.dropna(axis = 0, inplace = True)
df.head(3)

,<DTYYYYMMDD>,<OpenFixed>,<HighFixed>,<LowFixed>,<CloseFixed>,<Volume>,<Open>,<High>,<Low>,<Close>,<VolumeDeal>,<VolumeFB>,<VolumeFS>,datetime,Month_no,Year_no,Period,Daily_return
<Ticker>,,,,,,,,,,,,,,,,,,
SHS,20210428,26.9,27.9,26.7,27.3,4763896,26.9,27.9,26.7,27.3,0,100,5600,2021-04-28,4,2021,2021-4,-0.364964
SHS,20210427,26.9,27.3,26.5,26.9,4020723,26.9,27.3,26.5,26.9,40000,11000,30000,2021-04-27,4,2021,2021-4,-1.465201
SHS,20210426,28.3,29.0,27.0,27.0,5697714,28.3,29.0,27.0,27.0,0,13700,0,2021-04-26,4,2021,2021-4,0.371747


In [10]:
fig = go.Figure()
px.line(df, x='datetime', y='Daily_return', labels={'x':'Date', 'y':'Daily_Return_%'})

In [11]:
df['Daily_return'].describe()

count    951.000000
mean       0.026046
std        2.777738
min       -6.534091
25%       -1.357787
50%        0.000000
75%        1.148483
max       35.714286
Name: Daily_return, dtype: float64

In [12]:
fig = go.Figure()
px.histogram(df, x="Daily_return",histnorm='percent')
#ff.create_distplot(df, group_labels)

In [13]:
df['<Close>'].describe()

count    951.000000
mean      49.019085
std       22.484325
min       14.200000
25%       31.900000
50%       43.500000
75%       61.100000
max      118.000000
Name: <Close>, dtype: float64

In [14]:
dfig = go.Figure()
px.histogram(df.head(60), x="Daily_return",histnorm='percent')

In [15]:


df['Volumn'] = df['<Volume>']/1000000
Volumnshort_rolling = df['Volumn'].rolling(window=7).mean()

fig = go.Figure()
fig.update_layout(  autosize=False,   width=1200,    height=800)
fig.add_trace(go.Scatter(x=df.datetime, y=df['Daily_return'], 
                        mode='lines', name='Dailly_Retun'))
fig.add_trace(go.Scatter(x=df.datetime, y=df['Volumn'], 
                        mode='lines+markers', name='Volumn'))

fig.add_trace(go.Scatter(x=df.datetime, y=Volumnshort_rolling, 
                      mode ='lines', name='Span_7_Rolling_vol'))

fig.add_trace(go.Scatter(x=df.datetime, y=df['<Close>'], 
                        mode='lines+markers', name='Price'))

In [16]:
df['Volumn'].head(30).describe()

count    30.000000
mean      1.477090
std       0.348590
min       0.843500
25%       1.177975
50%       1.496550
75%       1.721300
max       2.150000
Name: Volumn, dtype: float64

In [17]:
df['Volumn'].head(10).describe()

count    10.000000
mean      1.330420
std       0.379352
min       0.843500
25%       1.114625
50%       1.227250
75%       1.450325
max       2.122800
Name: Volumn, dtype: float64

In [18]:
df.head(3)

,<Close>,datetime,<Volume>,Period,Year_no,Daily_return,Volumn
<Ticker>,,,,,,,
VCI,62.3,2021-04-27,843500,2021-17,2021,-2.656250,0.8435
VCI,61.0,2021-04-26,1024700,2021-17,2021,-2.086677,1.0247
VCI,62.2,2021-04-23,1100100,2021-16,2021,1.967213,1.1001
